# Tarea 2. NN with LMS, Problema 1

## Fundamentos de inteligencia computacional
## por: Juan Pablo Arango A. & Simón Zapata Caro

In [84]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from pydoc import help  # can type in the python console `help(name of function)` to get the documentation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import stats
from IPython.display import display, HTML
from pandas.tools.plotting import scatter_matrix
from matplotlib.artist import setp
import time

plt.rcParams['figure.figsize'] = (10,16)

In [85]:
df1     = pd.io.excel.read_excel("DatosPunto1.xlsx", sheetname=0)
classes = df1.iloc[:,-1].copy()
df1 = df1.iloc[:,0:-2].copy()
#normalización min-max
df1 = (df1-np.min(df1, axis = 0))/(np.max(df1, axis = 0)-np.min(df1, axis = 0))

y = np.array(classes)
X = np.array(df1)

In [86]:
X_original = X

Nsamples = X.shape[0]
Ntrain   = int(0.7*Nsamples)
Nval     = int(0.15*Nsamples)
Ntest    = Nsamples-Ntrain-Nval

X_test = X[(Nsamples-Ntest):Nsamples,:]
y_test = y[(Nsamples-Ntest):Nsamples  ]

Xv = X[0:(Nsamples-Ntest),:]
yv = y[0:(Nsamples-Ntest)  ]

In [87]:
def derivating(fx):
    difcoefs = [1,-1]
    return np.convolve(fx,difcoefs,mode='same')

def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

def clearList(L): #arroja una lista de L elementos vacios, no sabia de qué otra forma hacerla
    listica = []
    for i in range(0,L):
        listica.append([]);
    return listica

def LMS_training(X,y,iteraciones=500,alpha=0.1,de_min=0.01):
    
    ##falta mirar como imprimir Error en tiempo real
    
    w_          = np.random.rand(X.shape[1] + 1,)
    n           = X.shape[0]
    nf          = X.shape[1]
    sw          = 0
    j           = 0
    index       = []
    error_array = []
    

    while j < iteraciones and sw == 0:
        for i in range(0,n):
            g  = np.dot(w_,np.insert(X[i,:],0,1).T)
            e  = g - y[i]
            w_ = w_ - alpha*e*np.insert(X[i,:],0,1)
        predicted = []
        for i in range(0,n):
            predicted.append(Predict(X[i,:],w_))
        g = (y-predicted)
        E = 0.5*np.sum(g*g.T)/y.shape #queda la duda
        
        index.append(j)
        error_array.append(E[0])
        d_error_array = derivating(error_array)
        
        if np.abs(d_error_array[len(d_error_array)-1]) < de_min:
            sw = 1
        
        j = j + 1
    print('Función de error: '+str(E[0])+' iterando ' +str(j)+' veces, con alpha: '+str(alpha))
    
    return [w_,predicted]

def Predict(X,w_): 
    return np.dot(w_,np.insert(X,0,1).T)


### Entrenamiento de la red neuronal usando validación bootstraping para $\alpha = logspace(10^{-5}, 0.5=10^{-0.30102}, N = 30)$

In [5]:
index=np.random.permutation(X.shape[0])
X1    = Xv[index,:]
y1    = yv[index]

X_train = X1[0:Ntrain,:]
X_val   = X1[Ntrain:(Ntrain+Nval),:]
y_train = y1[0:Ntrain  ]
y_val   = y1[Ntrain:(Ntrain+Nval)  ]

alpha = np.logspace(-5,-0.30102,num=30)

In [6]:
wLMS       = clearList(len(alpha))
predicted  = clearList(len(alpha))
predicted2 = clearList(len(alpha))
ecm1       = clearList(len(alpha))
ecm2       = clearList(len(alpha))

for i in range(len(alpha)):
    [wLMS[i],predicted[i]] = LMS_training(X_train,y_train,iteraciones=1000,de_min=0.001,alpha=alpha[i])
    ecm1[i] = np.sqrt(np.sum(((predicted[i]-y_train)*(predicted[i]-y_train)))/y_train.shape)[0]
    
    p_i = []
    for j in range(0,X_val.shape[0]):
        p_i.append(Predict(X_val[j,:],wLMS[i]))
    predicted2[i] = p_i
    ecm2[i] = np.sqrt(np.sum(((predicted2[i]-y_val)*(predicted2[i]-y_val)))/y_val.shape)[0]



Función de error: 7.098578201791492 iterando 1000 veces, con alpha: 1e-05
Función de error: 6.673252349813438 iterando 1000 veces, con alpha: 1.4522246129615342e-05
Función de error: 6.282327593392731 iterando 917 veces, con alpha: 2.108956326491282e-05
Función de error: 6.150924584910585 iterando 738 veces, con alpha: 3.0626782849915814e-05
Función de error: 6.009747765591327 iterando 622 veces, con alpha: 4.447696787047604e-05
Función de error: 5.978808181176455 iterando 496 veces, con alpha: 6.459054745140466e-05
Función de error: 5.854437754941373 iterando 378 veces, con alpha: 9.379998277358975e-05
Función de error: 5.836716257990311 iterando 309 veces, con alpha: 0.00013621864367917524
Función de error: 5.7960507731477335 iterando 262 veces, con alpha: 0.00019782006709513538
Función de error: 5.718561825214475 iterando 219 veces, con alpha: 0.00028727917037325835
Función de error: 5.637871125464687 iterando 245 veces, con alpha: 0.00041719388200721575
Función de error: 5.55647859

In [7]:
print('para cada topologia de red')
d = {'alpha': alpha,'error de entrenamiento': ecm1, 'error de validación' : ecm2}
df = pd.DataFrame(data=d)
df

para cada topologia de red


,alpha,error de entrenamiento,error de validación
0,0.000010,3.767911,3.913930
1,0.000015,3.653287,3.835964
2,0.000021,3.544666,3.721764
3,0.000031,3.507399,3.673115
4,0.000044,3.466914,3.613401
5,0.000065,3.457979,3.600316
6,0.000094,3.421823,3.545076
7,0.000136,3.416641,3.531592
8,0.000198,3.404718,3.507562
9,0.000287,3.381882,3.469546


In [16]:
#utilizando la topologia nro 17 de la tabla anterior
prediction_testing = []
for j in range(0,X_test.shape[0]):
    prediction_testing.append(Predict(X_test[j,:],wLMS[17]))
ecm_testing = np.sqrt(np.sum(((prediction_testing-y_test)*(prediction_testing-y_test)))/y_test.shape)[0]
print('error de testeo: '+str(ecm_testing))

error de testeo: 3.347225903426367


### Entrenamiento de la red neuronal usando validación cruzada con 10 folds para $\alpha = logspace(10^{-4}, 10^{-1}, N = 10)$

In [90]:
X = Xv
y = yv
Nf = 10
N  = X.shape[0]
X_train = clearList(Nf)
y_train = clearList(Nf)
X_val   = clearList(Nf)
y_val   = clearList(Nf)

index = chunkIt(np.arange(N),Nf)
for k in range(Nf):
    i = [x for i,x in enumerate(np.arange(Nf)) if i!=k]
    index2 = []
    for j in range(len(i)):
        index2 += list(index[i[j]])
    X_val[k]   = X[list(index[k]),:]
    y_val[k]   = y[list(index[k])  ]
    X_train[k] = X[index2,  :]
    y_train[k] = y[index2    ]

In [107]:
alpha = np.logspace(-4,-1,num=10)
e_train = np.zeros((Nf,len(alpha)))
e_val   = np.zeros((Nf,len(alpha)))
#e_train[0,0]=5
minE_val   = 100.
for k in range(Nf):
    print('******* validación '+str(k+1)+' *******')
    wLMS       = clearList(len(alpha))
    predicted  = clearList(len(alpha))
    predicted2 = clearList(len(alpha))
    ecm1       = clearList(len(alpha))
    ecm2       = clearList(len(alpha))

    for i in range(len(alpha)):
        [wLMS[i],predicted[i]] = LMS_training(X_train[k],y_train[k],iteraciones=1000,de_min=0.001,alpha=alpha[i])
        ecm1[i] = np.sqrt(np.sum(((predicted[i]-y_train[k])*(predicted[i]-y_train[k])))/y_train[k].shape)[0]

        p_i = []
        for j in range(0,X_val[k].shape[0]):
            p_i.append(Predict(X_val[k][j,:],wLMS[i]))
        predicted2[i] = p_i
        ecm2[i] = np.sqrt(np.sum(((predicted2[i]-y_val[k])*(predicted2[i]-y_val[k])))/y_val[k].shape)[0]
        
        if(ecm2[i] < minE_val):
            wLMS_optimo = wLMS[i]
            minE_val = ecm2[i]
        
    e_train[k] = ecm1
    e_val[k]   = ecm2

******* validación 1 *******
Función de error: 5.700957356990323 iterando 396 veces, con alpha: 0.0001
Función de error: 5.563934068060788 iterando 271 veces, con alpha: 0.00021544346900318845
Función de error: 5.382845694256182 iterando 257 veces, con alpha: 0.00046415888336127773
Función de error: 5.25439008885269 iterando 189 veces, con alpha: 0.001
Función de error: 5.198613012992077 iterando 132 veces, con alpha: 0.002154434690031882
Función de error: 5.189276368001017 iterando 81 veces, con alpha: 0.004641588833612777
Función de error: 5.261159720688692 iterando 47 veces, con alpha: 0.01
Función de error: 5.498606957169293 iterando 23 veces, con alpha: 0.021544346900318822
Función de error: 6.203628014303937 iterando 14 veces, con alpha: 0.046415888336127774
Función de error: 8.257181676757796 iterando 13 veces, con alpha: 0.1
******* validación 2 *******
Función de error: 5.899645660819944 iterando 366 veces, con alpha: 0.0001
Función de error: 5.803536061521189 iterando 259 vec

Función de error: 5.866587815301896 iterando 25 veces, con alpha: 0.021544346900318822
Función de error: 6.844585914826473 iterando 13 veces, con alpha: 0.046415888336127774
Función de error: 7.051872327730335 iterando 330 veces, con alpha: 0.1


In [108]:
e_train

array([[3.37667214, 3.33584594, 3.2811113 , 3.24172488, 3.22447298,
        3.22157613, 3.24381249, 3.31620475, 3.52239351, 4.06378682],
       [3.43500965, 3.40691534, 3.35060609, 3.3171392 , 3.30333161,
        3.30918952, 3.34813662, 3.43285018, 3.62422953, 3.70665262],
       [3.47071999, 3.42236361, 3.37187824, 3.33635304, 3.32369011,
        3.3296    , 3.36674624, 3.4538069 , 3.65691065, 4.18182184],
       [3.39315241, 3.35458588, 3.30141249, 3.26476758, 3.24888587,
        3.25608958, 3.29545247, 3.38187935, 3.57922407, 3.62629054],
       [3.48331638, 3.44914136, 3.39201046, 3.35735616, 3.34427168,
        3.34923338, 3.38463455, 3.46436743, 3.46469807, 3.75701221],
       [3.45093059, 3.41040536, 3.35778044, 3.32082166, 3.30760116,
        3.31582109, 3.36057802, 3.45464036, 3.65029587, 3.73823055],
       [3.38917085, 3.34863477, 3.28532246, 3.24847674, 3.23515621,
        3.24337681, 3.29214367, 3.38735886, 3.60684713, 4.15054173],
       [3.4739401 , 3.43891545, 3.3790966

In [109]:
e_val

array([[3.95956544, 4.10404504, 4.21579119, 4.23120902, 4.22061836,
        4.23097742, 4.30807809, 4.4516403 , 4.70659973, 5.12307382],
       [3.50651682, 3.46946145, 3.39097657, 3.33351547, 3.30967252,
        3.3232532 , 3.37456637, 3.46720755, 3.69982157, 3.45696195],
       [3.32323293, 3.24060878, 3.15442546, 3.09664332, 3.07706587,
        3.0954655 , 3.16801612, 3.31820005, 3.61659603, 4.22429606],
       [3.85104271, 3.81854458, 3.76543004, 3.71945898, 3.7046894 ,
        3.72190503, 3.77675156, 3.87610778, 4.07417253, 4.12134409],
       [2.9414147 , 2.90083128, 2.84600328, 2.81373969, 2.81003564,
        2.83829683, 2.90916226, 2.98960653, 2.8591009 , 2.94946058],
       [3.26448096, 3.23003225, 3.18226147, 3.14434966, 3.1254107 ,
        3.1171358 , 3.13103251, 3.17787397, 3.30375728, 3.53018641],
       [3.88792758, 3.8427868 , 3.78994318, 3.7630194 , 3.74798616,
        3.73783024, 3.7458093 , 3.78667443, 3.98701058, 4.5407101 ],
       [3.04056418, 2.98852781, 2.9254210

In [111]:
#utilizando la topologia óptima
prediction_testing = []
for j in range(0,X_test.shape[0]):
    prediction_testing.append(Predict(X_test[j,:],wLMS_optimo))
ecm_testing = np.sqrt(np.sum(((prediction_testing-y_test)*(prediction_testing-y_test)))/y_test.shape)[0]
print('error de testeo: '+str(ecm_testing))

error de testeo: 3.1050278300587455
